# Data Science Capstone Project

Introduction

In [244]:
#Importing libraries
import pandas as pd
import numpy  as np
import re
import requests
from bs4 import BeautifulSoup as bs
import functools
import io
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# Credentials for APIs
APIKEY = '7d82a34ae444c110070dd93aa647e808eada15a9' #Papilite
CLIENT_ID = 'OEKMEBXITJT02VBZB3LP4ATYZ35OI3PLISMSZUTSJSFMBZQI' # Foursquare 
CLIENT_SECRET = 'POCWPILITAHTRCVJB1EKV1Y1OM30PZXOGVYAPZ4CCPZKHMUQ' # Foursquare   
VERSION = '20180605' # Foursquare
LIMIT = 20 # Foursquare

Data Collection

In [2]:
# Scraping for data
url = "https://www.postnummerservice.se/information/svenska-postnummer-och-postorter/stockholm/stockholm-kommun/"
html_content = requests.get(url).text
soup = bs(html_content, "lxml")
postal_codes_city_odd = soup.find_all("li", attrs={"class": "p5 odd"})
postal_codes_city_even = soup.find_all("li", attrs={"class": "p5 even"})
postal_codes_city = []
for x in postal_codes_city_odd:
    postal_codes_city.append((x.get_text()))
for y in postal_codes_city_even:
    postal_codes_city.append((y.get_text()))
#print(postal_codes)
#print(postal_codes_city)
#len(postal_codes_city)

In [90]:
# bot protected

# Scraping for data
#url = "https://postnummer.cybo.com/sverige/stockholms-l%C3%A4n/"
#html_content = requests.get(url).text
#soup = bs(html_content, "lxml")
#print(soup)
#print(postal_codes_city)
#len(postal_codes_city)

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]--><!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]--><!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Access denied | postnummer.cybo.com used Cloudflare to restrict access</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/main.css" id="cf_styles-css" media="screen,projection" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="cf-wrapper">
<div class="cf-alert cf-alert-error cf-cookie-error hidden" data-translate="enable_cookies" id="cookie-alert">Please enable cookies.</div>
<div class="p-0" id="cf-er

In [3]:
# Only request Papilite if not in cache
if "cached_requests_get_papilite" not in dir():
    @functools.lru_cache(maxsize=10000)
    def cached_requests_get_papilite(url):
            response = requests.get(url)
            return response

In [98]:
# Function for collecting data from papilite for city
def getPostalCodesCoordinates(city, current_postal_code):
    postal_codes_cords=pd.DataFrame()

    for POSTORT in city:
        # create the API request URL
        url_papilite = 'https://api.papapi.se/lite/?query={}&format=csv&apikey={}'.format(
            POSTORT,
            APIKEY)  
       
        # make the GET request
        response = cached_requests_get_papilite(url=url_papilite)

        # Clean up the response
        content = re.sub("###.*###","",response.text) #remove note at the end
        content = content.split('\n', 1)[1] # remove header
        buffer = io.StringIO(content) # enable read csv from buffer
        
        # Read response to dataframe, add header
        df_response = pd.read_csv(buffer, sep = ";", header=None, index_col=False, names=['postal code','city','latitude','longitude','county_code','county','state_code','state','note','updated'])
        postal_codes_cords = postal_codes_cords.append(df_response)
    
    # create url for current_postal_code
    url_papilite_current = 'https://api.papapi.se/lite/?query={}&format=csv&apikey={}'.format(
        current_postal_code,
        APIKEY)
    
    # make the GET request
    response = cached_requests_get_papilite(url=url_papilite_current)
    
    # Clean up the response
    content = re.sub("###.*###","",response.text) #remove note at the end
    content = content.split('\n', 1)[1] # remove header
    buffer = io.StringIO(content) # enable read csv from buffer
    
    # Read response to dataframe, add header
    df_response = pd.read_csv(buffer, sep = ";", header=None, index_col=False, names=['postal code','city','latitude','longitude','county_code','county','state_code','state','note','updated'])
    postal_codes_cords = postal_codes_cords.append(df_response)
    
    return(postal_codes_cords)

In [197]:
# Cleaning up results
postal_codes_cords = getPostalCodesCoordinates(city = postal_codes_city,current_postal_code='41757')                                
print("Number of rows before removing duplicates: ",postal_codes_cords.shape[0])
postal_codes_cords = postal_codes_cords = postal_codes_cords.drop_duplicates() 
print("Number of rows after removing duplicates: ",postal_codes_cords.shape[0])
postal_codes_cords= postal_codes_cords.drop(columns=['county_code','county','state_code','state','note','updated'])
postal_codes_cords.head(30)

Number of rows before removing duplicates:  1121
Number of rows after removing duplicates:  1111


,postal code,city,latitude,longitude
0,12804,Bagarmossen,59.274388,18.133849
1,12805,Bagarmossen,59.274388,18.133849
2,12841,Bagarmossen,59.278837,18.125338
3,12842,Bagarmossen,59.274137,18.126981
4,12843,Bagarmossen,59.275399,18.131139
5,12844,Bagarmossen,59.272635,18.142589
6,12845,Bagarmossen,59.276074,18.136789
7,12846,Bagarmossen,59.277752,18.135467
8,12847,Bagarmossen,59.282704,18.134656
9,12848,Bagarmossen,59.278010,18.146599


In [198]:
# Only request Foursquare if not in cache
if "cached_requests_get_foursquare" not in dir():
    @functools.lru_cache(maxsize=10000)
    def cached_requests_get_foursquare(url):
            response = requests.get(url)
            return response

In [224]:
# Get nearby venues from foursquare
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    global venues_list
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        request_json = cached_requests_get_foursquare(url).json()
        #print(request_json)
        results = request_json["response"]['groups'][0]['items']
   
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code',
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [231]:
# Collect information from foursquare, create dummies
postal_codes_venues = getNearbyVenues(names=postal_codes_cords['postal code'],
                                   latitudes=postal_codes_cords['latitude'],
                                   longitudes=postal_codes_cords['longitude']
                                    )
postal_codes_venues.head()

postal_codes_dummies = pd.get_dummies(postal_codes_venues[['Venue Category']], prefix="", prefix_sep="")
postal_codes_dummies.insert(0,'postal code',postal_codes_venues['Postal Code'])

postal_codes_grouped = postal_codes_dummies.groupby(['postal code'],sort=False).mean().reset_index()
#postal_codes_grouped.head()
#len(postal_codes_grouped)
#postal_codes_grouped.describe()
#print(postal_codes_grouped)

postal_codes_merged = postal_codes_cords.merge(postal_codes_grouped,on='postal code', how='outer')
postal_codes_merged.fillna(0,inplace=True)  
postal_codes_grouped.head(20)
#postal_codes_venues.head(30)
postal_codes_merged.head(20)


,postal code,city,latitude,longitude,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Track Stadium,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,12804,Bagarmossen,59.274388,18.133849,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,12805,Bagarmossen,59.274388,18.133849,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,12841,Bagarmossen,59.278837,18.125338,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,12842,Bagarmossen,59.274137,18.126981,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,12843,Bagarmossen,59.275399,18.131139,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5,12844,Bagarmossen,59.272635,18.142589,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
6,12845,Bagarmossen,59.276074,18.136789,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
7,12846,Bagarmossen,59.277752,18.135467,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
8,12847,Bagarmossen,59.282704,18.134656,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
9,12848,Bagarmossen,59.278010,18.146599,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [232]:
# sort in descending order function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [233]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['postal code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postal_codes_venues_sorted = pd.DataFrame(columns=columns)
postal_codes_venues_sorted['postal code'] = postal_codes_merged['postal code']

for ind in np.arange(postal_codes_grouped.shape[0]):
    postal_codes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(postal_codes_grouped.iloc[ind, :], num_top_venues)

postal_codes_venues_sorted.insert(1,'city',postal_codes_merged['city'])
postal_codes_venues_sorted.insert(2,'latitude',postal_codes_merged['latitude'])
postal_codes_venues_sorted.insert(3,'longitude',postal_codes_merged['longitude'])
postal_codes_venues_sorted.head()
#duplicates = postal_codes_venues_sorted[postal_codes_venues_sorted['latitude'].duplicated()]
#duplicates.tail(30)
#postal_codes_venues.loc[postal_codes_venues['Postal Code']== 10005]
#postal_codes_venues.head()


,postal code,city,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,12804,Bagarmossen,59.274388,18.133849,Supermarket,Park,Grocery Store,Pizza Place,Fast Food Restaurant,Church,Sushi Restaurant,Bakery,Japanese Restaurant,Pub
1,12805,Bagarmossen,59.274388,18.133849,Supermarket,Park,Grocery Store,Pizza Place,Fast Food Restaurant,Church,Sushi Restaurant,Bakery,Japanese Restaurant,Pub
2,12841,Bagarmossen,59.278837,18.125338,Sushi Restaurant,Bakery,Pub,Athletics & Sports,Scandinavian Restaurant,Bus Stop,Fast Food Restaurant,Supermarket,Tennis Court,Grocery Store
3,12842,Bagarmossen,59.274137,18.126981,Grocery Store,Fast Food Restaurant,Church,Bus Station,Bakery,Café,Pub,Sushi Restaurant,Supermarket,Other Great Outdoors
4,12843,Bagarmossen,59.275399,18.131139,Grocery Store,Park,Sushi Restaurant,Supermarket,Fast Food Restaurant,Other Great Outdoors,Pub,Church,Pharmacy,Bakery


In [234]:
# Clustering

# set number of clusters
kclusters = 10

postal_codes_clustering = postal_codes_merged.drop(['postal code','city','latitude','longitude'], axis=1)
#postal_codes_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(postal_codes_clustering)

# check cluster labels generated for each row in the dataframe
cluster = kmeans.labels_
#print(cluster[0:100])
postal_codes_clustered_dummies = postal_codes_merged
postal_codes_clustered = postal_codes_venues_sorted
postal_codes_clustered_dummies.insert(4,'cluster',cluster)
postal_codes_clustered.insert(4,'cluster',cluster)

In [253]:
# Map clusters
latitude = postal_codes_clustered['latitude'].get_value(1)
longitude = postal_codes_clustered['longitude'].get_value(1) 
#print(latitude,longitude)

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters+1)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, city, cluster in zip(postal_codes_clustered['latitude'], postal_codes_clustered['longitude'], postal_codes_clustered['postal code'], postal_codes_clustered['city'], postal_codes_clustered['cluster']):
    label = folium.Popup(str(poi) +' '+str(city)+' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

C:\Users\alexg\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\alexg\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


NameError: name 'Toronto_merged' is not defined